# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [48]:
# Load libraries
%load_ext dotenv
%dotenv 
import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
import dask.dataframe as dd
import pandas as pd
import numpy as np

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [49]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/forest+fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [50]:
# Created a dataframe X
X = fires_dt.drop(columns=['area'])
print(X.head())

   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain
0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0
1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0
2        7        4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0
3        8        6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2
4        8        6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0


In [51]:
## Create a dataframe Y
Y = fires_dt['area']
print(Y.head())

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: area, dtype: float64


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [55]:
# Load the libraries
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [57]:
# determine Numerical and Categorical features

num_columns = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
cat_columns = ['month', 'day']

In [58]:
# Divide data for train and test sets for Pipeline_A
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
## Preproc 1

# # Implement scaling for numerical features for preproc1
scale1 = StandardScaler()

# Implement OneHotEncoder for categorical features for preproc1
all_data = pd.concat([X_train[['month', 'day']], X_test[['month', 'day']]], axis=0) # attempt to escape errors
onehot_encoder_1 = OneHotEncoder(handle_unknown='ignore', drop='first')
onehot_encoder_1.fit(all_data)

numerical_imputer = SimpleImputer(strategy='mean')  
categorical_imputer = SimpleImputer(strategy='most_frequent') 

# Create ColumnTransformer for preproc1
preproc1 = ColumnTransformer(
    transformers=[
        ('numer', Pipeline([
            ('imputer', numerical_imputer),
            ('scaler', scale1)
        ]), num_columns),
        
        ('category', Pipeline([
            ('imputer', categorical_imputer), 
            ('onehot', onehot_encoder_1)
        ]), cat_columns)
    ]
)
preproc1

ColumnTransformer(transformers=[('numer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('category',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [63]:
## Preproc 2

# Implement scaling for numerical features for preproc2
scale2 = RobustScaler()

# Implement OneHotEncoder for categorical features for preproc1
all_data = pd.concat([X_train[['month', 'day']], X_test[['month', 'day']]], axis=0)
onehot_encoder_2 = OneHotEncoder(handle_unknown='ignore', drop='first')
onehot_encoder_2.fit(all_data)

# Implement non-linear transformation of the numerical numbers
sqrt_transform = FunctionTransformer(lambda x: np.sqrt(np.maximum(x + 1, 0)), validate=True)

# Create ColumnTransformer for preproc2
preproc2 = ColumnTransformer(
    transformers=[
        ('numer', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')), 
            ('scaler', scale2),  
            ('sqrt', sqrt_transform)   
        ]), num_columns),
        ('category', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')), 
            ('onehot', onehot_encoder_2)
        ]), cat_columns)
])  
preproc2

ColumnTransformer(transformers=[('numer',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', RobustScaler()),
                                                 ('sqrt',
                                                  FunctionTransformer(func=<function <lambda> at 0x158fe3b80>,
                                                                      validate=True))]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain']),
                                ('category',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [66]:
# Import necessary libraries 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer

In [67]:
# Pipeline A = preproc1 + baseline

Pipeline_A = Pipeline([
    ('preprocessing', preproc1), 
    ('imputer', SimpleImputer(strategy='mean')),
    ('regressor', LinearRegression())
])
Pipeline_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('imputer', SimpleImputer()),
                ('regressor', LinearRegression())])

In [68]:
# Fit train sets
Pipeline_A.fit(X_train, Y_train)

# Evaluate model with cross-validation

scoring = ['neg_mean_squared_error', 'r2']

cross_valid = cross_validate(Pipeline_A, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)
print(cross_valid)


{'fit_time': array([0.02032709, 0.01002216, 0.01071   , 0.00993204, 0.0104599 ]), 'score_time': array([0.00765085, 0.00584698, 0.00573301, 0.00658298, 0.00557995]), 'test_neg_mean_squared_error': array([-1662.74206816,  -383.3025444 , -1113.39177827, -7297.38737126,
        -716.99134725]), 'train_neg_mean_squared_error': array([-2077.04883453, -2390.52775392, -2274.26383551,  -730.90862171,
       -2304.64275901]), 'test_r2': array([-0.01926665, -0.77182861, -0.54567613, -0.01285267, -0.22123044]), 'train_r2': array([0.05253757, 0.05984555, 0.06080771, 0.08034482, 0.0573024 ])}


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [69]:
# Pipeline B = preproc2 + baseline

Pipeline_B = Pipeline([
    ('preprocessing', preproc2), 
    ('imputer', SimpleImputer(strategy='mean')),
    ('regressor', LinearRegression())
])
Pipeline_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('sqrt',
                                                                   FunctionTransformer(func=<function <lambda> at 0x158fe3b80>,
                                                                                       validate=True))]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('imputer', SimpleImputer()),
                ('regressor', LinearRegression())])

In [80]:
# Fit train sets
Pipeline_B.fit(X_train, Y_train)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('sqrt',
                                                                   FunctionTransformer(func=<function <lambda> at 0x158fe3b80>,
                                                                                       validate=True))]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('imputer', SimpleImputer()),
                ('regressor', LinearRegression())])

In [81]:
# Evaluate model with cross-validation
scoring_2 = ['neg_mean_squared_error', 'r2']

cross_valid_2 = cross_validate(Pipeline_B, X_train, Y_train, cv=5, scoring=scoring_2, return_train_score=True)
print(cross_valid_2)

{'fit_time': array([0.03516102, 0.01286125, 0.01192713, 0.01229191, 0.012429  ]), 'score_time': array([0.01055431, 0.00454092, 0.00510979, 0.00516725, 0.00483775]), 'test_neg_mean_squared_error': array([-1640.94962499,  -366.06614562,  -950.54708189, -7288.08406349,
        -711.56555803]), 'train_neg_mean_squared_error': array([-2092.3759303 , -2396.38598307, -2280.77985111,  -734.1434889 ,
       -2308.6236282 ]), 'test_r2': array([-0.00590781, -0.69215279, -0.31960552, -0.0115614 , -0.21198885]), 'train_r2': array([0.04554599, 0.05754161, 0.05811682, 0.07627459, 0.05567405])}


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [90]:
# Pipeline C = preproc1 + advanced model

# Import libraries for advanced model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# define value for Forest Regression
advanced = RandomForestRegressor(n_estimators=100, random_state=42)

Pipeline_C = Pipeline([
    ('preprocessing', preproc1), 
    ('model', advanced)  
])
Pipeline_C

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [82]:
# Train Pipeline_C
Pipeline_C.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [83]:
# Make prediction
Y_pred = Pipeline_C.predict(X_test)

# Model evaluation with mean_squared_error
mse = mean_squared_error(Y_test, Y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 11971.070578327131


In [84]:
# Pipeline D = preproc2 + advanced model

Pipeline_D = Pipeline([
    ('preprocessing', preproc2), 
    ('model', advanced)  
])
Pipeline_D   

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('sqrt',
                                                                   FunctionTransformer(func=<function <lambda> at 0x158fe3b80>,
                                                                                       validate=True))]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [85]:
# Train Pipeline_D
Pipeline_D.fit(X_train, Y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('sqrt',
                                                                   FunctionTransformer(func=<function <lambda> at 0x158fe3b80>,
                                                                                       validate=True))]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [86]:
# Make prediction
Y_pred_2 = Pipeline_D.predict(X_test)

# Model evaluation with mean_squared_error
mse_2 = mean_squared_error(Y_test, Y_pred_2)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 11971.070578327131


# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [87]:
# Import library
from sklearn.model_selection import GridSearchCV
import warnings

In [ ]:
# For Pipeline_A
# Define the parameters for Pipeline_A
param_grid_Pipeline_A = {
    'preprocessing__numer__imputer__strategy': ['mean', 'median', 'most_frequent'],
    'preprocessing__category__imputer__strategy': ['mean', 'most_frequent'],
    'preprocessing__category__onehot__handle_unknown': ['ignore', 'error'],
    'preprocessing__category__onehot__drop': [None, 'first'],
    'regressor': [LinearRegression()]
}
# Run GridSearchCV
grid_search_Pipeline_A = GridSearchCV(Pipeline_A, param_grid_Pipeline_A, cv=5)

# Fit model with Pipeline_A
grid_search_Pipeline_A.fit(X_train, Y_train)

# Get the best result for Pipeline_A
print(grid_search_Pipeline_A.best_params_)

In [ ]:
# For Pipeline_B
# Define the parameters for Pipeline_B
param_grid_Pipeline_B = {
    'preprocessing__numer__imputer__strategy': ['mean', 'median', 'most_frequent'],  
    'preprocessing__numer__scaler': [StandardScaler()],
    'preprocessing__category__imputer__strategy': ['mean', 'most_frequent'],  
    'preprocessing__category__onehot__handle_unknown': ['ignore', 'error'],
    'preprocessing__category__onehot__drop': [None, 'first'],
    'regressor': [LinearRegression()]
}
# Run GridSearchCV    
grid_search = GridSearchCV(Pipeline_B, param_grid_Pipeline_B, cv=5, error_score='raise')

# Fit model with Pipeline_B
grid_search_Pipeline_B.fit(X_train, Y_train)

# Get the best result for Pipeline_B
print(grid_search_Pipeline_B.best_params_)

In [90]:
#For Pipeline_C
#Define the parameters for Pipeline_C
param_grid_Pipeline_C = {
    'model__n_estimators': [50, 100, 200],  
    'model__max_depth': [None, 10, 20, 30], 
    'model__min_samples_split': [2, 5, 10],  
    'model__min_samples_leaf': [1, 2, 4]  
}

# Run GridSearchCV
grid_search_Pipeline_C = GridSearchCV(Pipeline_C, param_grid_Pipeline_C, cv=5, verbose=1, n_jobs=-1)

# Fit model with Pipeline_C
grid_search_Pipeline_C.fit(X_train, Y_train)

# Return best parametrs
print("Best parameters:", grid_search_Pipeline_C.best_params_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'model__max_depth': 10, 'model__min_samples_leaf': 4, 'model__min_samples_split': 10, 'model__n_estimators': 200}


In [ ]:
#For Pipeline_D
#Define the parameters for Pipeline_D
param_grid_Pipeline_D = {
    'model__n_estimators': [50, 100, 200],  
    'model__max_depth': [None, 10, 20, 30], 
    'model__min_samples_split': [2, 5, 10],  
    'model__min_samples_leaf': [1, 2, 4]  
}

# Run GridSearchCV
grid_search_Pipeline_D = GridSearchCV(Pipeline_D, param_grid_Pipeline_D, cv=5, verbose=1, n_jobs=-1)

# Fit model with Pipeline_D
grid_search_Pipeline_D.fit(X_train, Y_train)

# Return best parametrs
print("Best parameters:", grid_search_Pipeline_D.best_params_)


# Evaluate

+ Which model has the best performance?

Model with Pipeline_C - because there is not errors. This is not the right answer, but I couldn't solve the problem with error- appearances. They are not critical, but still affect to the model.

# Export

+ Save the best performing model to a pickle file.

In [93]:
import pickle

with open('best_pipeline.pkl', 'wb') as file:
    pickle.dump(Pipeline_C, file)

with open('best_pipeline.pkl', 'rb') as file:
    loaded_pipeline = pickle.load(file)
loaded_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain']),
                                                 ('category',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['month', 'day'])])),
                ('model', RandomForestRegressor(random_state=42))])

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [ ]:
import shap

explainer = shap.Explainer(loaded_pipeline)

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.